In [66]:
%matplotlib inline
import geopandas as gpd
import pandas as pd
import folium
import branca.colormap as cm

In [67]:
fema = gpd.read_file("Data\\FEMA_6\\FEMA_6.shp")
fema = fema.sort_values(by='CTFIPS')

In [68]:
def ag_count(state):
    data = gpd.read_file("Data/SoVI2010_" + state + "/SoVI0610_" + state + ".shp")
    
    ## Creating CTFIPS, which FEMA data uses for counties
    data['CTFIPS'] = data['STATEFP10'] + data['COUNTYFP10']
    
    ## Aggregating SOVI data from block to county level
    data = data.groupby('CTFIPS').agg({"SOVI0610" + state: "mean", "P0010001": "sum"})
   
    ## Combining fema and sovi data for each county
    data = fema.merge(data, on="CTFIPS", how="right")
    
    ## Renaming the columns to things that make sense
    data = data.rename(columns={
        "SOVI0610" + state: "SOVI",
        "P0010001": "POPULATION",
        "TOTAL_DECL": "TOTAL_DECLARATIONS"
    })
    
    data = data.sort_values(by='CTFIPS')
    
    ## Creating a scale of the SOVI and FEMA data from 1 - number of counties in a state
    data['SOVI_SCORE'] = data['SOVI'].rank()
    data['FEMA_SCORE'] = data['TOTAL_DECLARATIONS'].rank()
    sovi = []
    fema_ranks = []
    rank = []
    n = len(data)
    
    ## Taking the SOV_SCORE and FEMA_SCORE, assign a SOVI_RANK and FEMA_RANK
    ## Rank = the correlation of SOV_SCORE and FEMA_SCORE, matching highs and lows and finding where it is different
    for index, row in data.iterrows():
        if row.SOVI_SCORE < (n/3):
            ## Low - Low = 1
            if row.FEMA_SCORE < (n/3):
                rank.append(1)
                fema_ranks.append(1)
            ## Low - High = 2
            elif row.FEMA_SCORE > (2 * (n/3)):
                rank.append(2)
                fema_ranks.append(3)
            ## Medium, don't care for rank
            else:
                rank.append(-1)
                fema_ranks.append(2)
            sovi.append(1)
        elif row.SOVI_SCORE > (2 * (n/3)):
            ## High - High = 4
            if row.FEMA_SCORE > (2 * (n/3)):
                rank.append(4)
                fema_ranks.append(3)
            ## High - Low = 3
            elif row.FEMA_SCORE < (n/3):
                rank.append(3)
                fema_ranks.append(1)
            ## Medium, don't care for rank
            else:
                rank.append(-1)
                fema_ranks.append(2)
            sovi.append(3)
        else:
            rank.append(-1)
            sovi.append(2)
            if row.FEMA_SCORE > (2 * (n/3)):
                fema_ranks.append(3)
            elif row.FEMA_SCORE < (n/3):
                fema_ranks.append(1)
            else:
                fema_ranks.append(2)
    ## Attaching the ranks to dataframe
    data['RANK'] = rank
    data['SOVI_RANK'] = sovi
    data['FEMA_RANK'] = fema_ranks
    return data

CA = ag_count("CA")
FL = ag_count("FL")
IN = ag_count("IN")
LA = ag_count("LA")
ME = ag_count("ME")
WA = ag_count("WA")
#CA.head()

,OBJECTID,STFIPS,ST_ABBR,CTFIPS,STATE,COUNTY,GEO_SUFFIX,TOTAL_DECLARATIONS,FLOOD_1964,ICE_STORM_,...,Shape_Le_1,Shape_Area,geometry,POPULATION,SOVI,SOVI_SCORE,FEMA_SCORE,RANK,SOVI_RANK,FEMA_RANK
0,58,06,CA,06001,California,Alameda,County,14,3,0,...,3.610286,0.196849,(POLYGON ((-121.9673689779958 37.4676361297038...,1510271.0,-0.174812,19.0,42.0,-1,2,3
1,39,06,CA,06003,California,Alpine,County,6,3,0,...,2.174212,0.199044,POLYGON ((-119.5854370003171 38.71321199963438...,1175.0,0.686160,27.0,5.0,-1,2,1
2,21,06,CA,06005,California,Amador,County,9,3,0,...,2.909742,0.161954,POLYGON ((-120.0723920002905 38.70276699999727...,38091.0,1.558083,42.0,18.0,3,3,1
3,53,06,CA,06007,California,Butte,County,10,3,0,...,4.205838,0.455942,POLYGON ((-121.0766949997742 39.59726399983401...,220000.0,1.954188,48.0,25.5,-1,3,2
4,22,06,CA,06009,California,Calaveras,County,8,1,0,...,2.963056,0.276236,POLYGON ((-120.0199510002383 38.43352099969468...,45578.0,1.929986,47.0,10.5,3,3,1


In [69]:
## Combining all of the states into one dataframe
## The rank columns don't show because there is 52 columns, so they are hidden
sovi = CA.append([FL, IN, LA, ME, WA])
sovi = sovi.sort_values(by='CTFIPS')
#sovi.head()

,COASTAL_19,COASTAL_2K,COUNTY,CTFIPS,DROUGHT_19,DROUGHT_2K,EARTHQUAKE,EARTHQUA_1,FEMA_RANK,FEMA_SCORE,...,TORNADO_19,TORNADO_2K,TOTAL_DECLARATIONS,TOTAL_DE_1,TSUNAMI_19,TSUNAMI_2K,TYPHOON_19,TYPHOON_2K,Winter_196,geometry
0,0,0,Alameda,06001,0,0,1,0,3,42.0,...,0,0,14,2,0,0,0,0,1,(POLYGON ((-121.9673689779958 37.4676361297038...
1,0,0,Alpine,06003,0,0,0,0,1,5.0,...,0,0,6,1,0,0,0,0,0,POLYGON ((-119.5854370003171 38.71321199963438...
2,0,0,Amador,06005,0,0,0,0,1,18.0,...,0,0,9,2,0,0,0,0,0,POLYGON ((-120.0723920002905 38.70276699999727...
3,0,0,Butte,06007,0,0,0,0,2,25.5,...,0,0,10,1,0,0,0,0,1,POLYGON ((-121.0766949997742 39.59726399983401...
4,0,0,Calaveras,06009,0,0,0,0,1,10.5,...,0,0,8,3,0,0,0,0,0,POLYGON ((-120.0199510002383 38.43352099969468...


In [70]:
## Creating dictionaries for the ranks that will be used be the color functions
sovi = sovi.set_index("CTFIPS")
sovi_dict = sovi["SOVI_RANK"]
fema_dict = sovi["FEMA_RANK"]
rank_dict = sovi['RANK']
#sovi.head()

,COASTAL_19,COASTAL_2K,COUNTY,DROUGHT_19,DROUGHT_2K,EARTHQUAKE,EARTHQUA_1,FEMA_RANK,FEMA_SCORE,FIRE_1964,...,TORNADO_19,TORNADO_2K,TOTAL_DECLARATIONS,TOTAL_DE_1,TSUNAMI_19,TSUNAMI_2K,TYPHOON_19,TYPHOON_2K,Winter_196,geometry
CTFIPS,,,,,,,,,,,,,,,,,,,,,
06001,0,0,Alameda,0,0,1,0,3,42.0,2,...,0,0,14,2,0,0,0,0,1,(POLYGON ((-121.9673689779958 37.4676361297038...
06003,0,0,Alpine,0,0,0,0,1,5.0,0,...,0,0,6,1,0,0,0,0,0,POLYGON ((-119.5854370003171 38.71321199963438...
06005,0,0,Amador,0,0,0,0,1,18.0,0,...,0,0,9,2,0,0,0,0,0,POLYGON ((-120.0723920002905 38.70276699999727...
06007,0,0,Butte,0,0,0,0,2,25.5,0,...,0,0,10,1,0,0,0,0,1,POLYGON ((-121.0766949997742 39.59726399983401...
06009,0,0,Calaveras,0,0,0,0,1,10.5,2,...,0,0,8,3,0,0,0,0,0,POLYGON ((-120.0199510002383 38.43352099969468...


In [71]:
## Color functions, creating step color schemes
sovi_color = cm.StepColormap(
    ['green', 'yellow', 'red'],
    vmin=1, vmax=3
)

fema_color = cm.StepColormap(
    ['green', 'yellow', 'red'],
    vmin=1, vmax=3
)

rank_color = cm.StepColormap(
    ['gray','green', 'yellow', 'orange', 'red'],
    vmin=-1, vmax=4
)

In [72]:
m = folium.Map(location=[40, -100], zoom_start=4, tiles='cartodbpositron',)

In [73]:
## Creating layer for the SOVI_RANK data
folium.GeoJson(
    sovi,
    style_function=lambda feature: {
        'fillColor': sovi_color(sovi_dict[feature['id']]),
        'color': 'gray',
        'weight': 1,
    },
    name = "SOVI",
    show = True,
    tooltip = folium.features.GeoJsonTooltip(
        fields = ['SOVI_RANK'],
        localize =True
    )
).add_to(m)

In [74]:
## Creating layer for the FEMA_RANK data
folium.GeoJson(
    sovi,
    style_function=lambda feature: {
        'fillColor': fema_color(fema_dict[feature['id']]),
        'color': 'gray',
        'weight': 1,
    },
    name = "FEMA",
    show = False,
    tooltip = folium.features.GeoJsonTooltip(
        fields = ['FEMA_RANK'],
        localize =True
    )
).add_to(m)

In [75]:
## Creating layer for the RANK data
folium.GeoJson(
    sovi,
    style_function=lambda feature: {
        'fillColor': rank_color(rank_dict[feature['id']]),
        'color': 'gray',
        'weight': 1,
    },
    name = "RANK",
    show = False,
    tooltip = folium.features.GeoJsonTooltip(
        fields = ['RANK'],
        localize =True
    )
).add_to(m)

In [76]:
## Output of map
folium.LayerControl(collapsed=False).add_to(m)

m.save(outfile='map.html')